<a href="https://colab.research.google.com/github/jdowner212/cs577_addernet/blob/main/AdderNet_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import torch
import tensorflow as tf
import tensorflow.keras.backend as K
import skimage


In [5]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
#tf.compat.v1.enable_eager_execution()


In [6]:
from torch.torch_version import TorchVersion
import numpy as np
import torch

def L1(a,b):
    return -1*np.abs(a-b)

def hard_tanh(value):
    if -1 < value and value < 1:
        return value
    elif value > 1:
        return 1
    elif value < -1:
        return -1

'''Equation 1'''
# modified by Equation 2


'''Equation 2'''
def Y_adder(X, F, m, n, t, similarity_f=L1):
    # F.shape returns (#filters, #channels, #rows, #columns)
    # X.shape returns (#channels, #rows, #columns)
    # t specifies filter #
    sum_ = 0
    _, c_in, d, _ = F.shape     
    for k in range(c_in):
        for j in range(d):
            for i in range(d):
                sum_ += similarity_f(X[k, m+i, n+j], F[i, j, k, t])
    return sum_

'''Equation 3'''
# ignore -- CNN formula

'''Equation 4'''
# ignore -- updated with equation 5

'''Equation 5'''
def dY_dF(X,F,m,n,i,j,k,t):
    X_ = X[k,m+i,n+j]
    F_ = F[t,k,i,j]
    return X_ - F_

'''Equation 6'''
def dY_dX(X,F,m,n,i,j,k,t):
    # clipped, full-precision gradient
    X_ = X[k,m+i,n+j]
    F_ = F[t,k,i,j]
    return hard_tanh(X_ - F_)

'''Equation 7'''
# ignore -- hard_tanh implemented previously

'''Equation 8'''
# ignore -- CNN formula

'''Equation 9'''
# def var_Y_adder(X,F,variance_f=torch.var):
    # check torch.var documentation: https://pytorch.org/docs/stable/generated/torch.var.html
    # not sure if we can call torch.var(X) with default parameters
    # or if we need to specify. Does this output a scalar or a tensor?
# Trying K.var as tensorflow substitute for torch.var -- make sure they work the same
# or tf.var?
def var_Y_adder(X,F,variance_f=K.var):
    var_X = variance_f(X)
    var_F = variance_f(F)
    ###
    _, c_in, d, _ = F.shape
    pi = np.pi

    return np.sqrt(pi/2)*(d**2)*(c_in)*(var_X + var_F)

'''Equation 10'''
def batch_norm(minibatch, gamma, beta):
    m = len(minibatch)
    mean = (1/m)*sum(minibatch)
    std = (1/m)*sum([(x_i-mean)**2 for x_i in minibatch])
    gamma*(minibatch-mean)/std + beta
    return gamma*(minibatch-mean)/std + beta

'''Equation 11'''
def dL_dMinibatch_i(minibatch,dL_dy,i,L,gamma):
    # In dL_dy, y is the result of applying batch_norm to the minibatch
    m = len(minibatch)
    mean = (1/m)*sum(minibatch)
    std = (1/m)*sum([(x_i-mean)**2 for x_i in minibatch])
    
    sum_ = 0
    for j in range(m):
        x_term = (minibatch[i]-minibatch[j])*(minibatch[j]-mean)/std
        sum_ += (dL_dy[i] - dL_dy[j]*(1 + x_term))
    sum_ *= gamma/((m**2)*std)
    
    return sum_

'''Equation 12'''
def delta_F_l(adaptive_lr_l, dL_dF_l, gamma):
    # the update delta for the filter in layer l
    return gamma*adaptive_lr_l*dL_dF_l

'''Equation 13'''
def adaptive_lr_l(dL_dF_l, eta, k):
    # k = number of elements in F_l -- I think equal to len(dL_dF_1)
    # in which case we don't need to explicitly provide it
    
    
    # l2_norm = torch.sqrt([g**2 for g in dL_dF_l])
    l2_norm = K.sqrt([g**2 for g in dL_dF_l]) # make sure torch.sqrt and K.sqrt are equivalents
    
    
    return eta*np.sqrt(k)/l2_norm

In [7]:
# # t specifies filter #
# sum_ = 0
# _, c_in, d, _ = F.shape     
# for k in range(c_in):
#     for j in range(d):
#         for i in range(d):
#             sum_ += similarity_f(X[k, m+i, n+j], F[i, j, k, t])
# return sum_


class adder(Function):
    @staticmethod
    def forward(ctx, W_col, X_col):
        ctx.save_for_backward(W_col,X_col)
        output = -(W_col.unsqueeze(2)-X_col.unsqueeze(0)).abs().sum(1)
        return output

    @staticmethod
    def backward(ctx,grad_output):
        W_col,X_col = ctx.saved_tensors
        grad_W_col = ((X_col.unsqueeze(0)-W_col.unsqueeze(2))*grad_output.unsqueeze(1)).sum(2)
        grad_W_col = grad_W_col/grad_W_col.norm(p=2).clamp(min=1e-12)*math.sqrt(W_col.size(1)*W_col.size(0))/5
        grad_X_col = (-(X_col.unsqueeze(0)-W_col.unsqueeze(2)).clamp(-1,1)*grad_output.unsqueeze(1)).sum(0)
        
        return grad_W_col, grad_X_col

NameError: ignored

In [8]:
def my_adder2d_function(X, W, padding='VALID', stride=1, k_size=3):
    if padding=='VALID':
        p=0
    else:
        p=1
    #W,X = F_tf, X_tf
    n_filters, d_filter, h_filter, w_filter = W.shape
    n_x, d_x, h_x, w_x = X.shape


    h_out = (h_x - h_filter + 2 * p) / stride + 1
    w_out = (w_x - w_filter + 2 * p) / stride + 1

    h_out, w_out = int(h_out), int(w_out)

    X_col = tf.image.extract_patches(np.rollaxis(X_rand,1,4), 
                                    sizes=[1,k_size,k_size,1], 
                                    strides=[1,stride,stride,1], 
                                    rates=[1,1,1,1], 
                                    padding='VALID') #'VALID' = 0, 'SAME' = however many to not change shape

    X_col = X_col.reshape((np.prod(X_col.shape[:-1]),X_col.shape[-1]))

    W_col = W.reshape(n_filters,-1)

    out = adder2d.apply(W_col,X_col)
    out = out.view(n_filters, h_out, w_out, n_x)
    out = out.permute(3, 0, 1, 2).contiguous()

In [9]:
tf.custom_gradient
def custom_op(X,activation,W):
    #staticmethod
    #def forward(ctx, W_col, X_col):
        #ctx.save_for_backward(W_col,X_col)
    output = -(W_col.unsqueeze(2)-X_col.unsqueeze(0)).abs().sum(1)
    if activation:
        output = activation(output)


    # def backward(ctx,upstream_gradient):
    def backward(upstream_gradient):
        #W_col,X_col = ctx.saved_tensors
        grad_W_col = ((X_col.unsqueeze(0)-W_col.unsqueeze(2))*upstream_gradient.unsqueeze(1)).sum(2)
        grad_W_col = grad_W_col/grad_W_col.norm(p=2).clamp(min=1e-12)*math.sqrt(W_col.size(1)*W_col.size(0))/5
        grad_X_col = (-(X_col.unsqueeze(0)-W_col.unsqueeze(2)).clamp(-1,1)*upstream_gradient.unsqueeze(1)).sum(0)
        
        return grad_W_col, grad_X_col

In [10]:
@tf.custom_gradient
def custom_op(inputs, activation, weights, biases):
    
    # forward computation
    z = tf.matmul(inputs, weights) + biases
    if activation is not None:
        z = activation(result)
    
    # backward computation
    def grad(upstream):
        inputs_gradient = tf.matmul(upstream, tf.transpose(weights))
        weights_gradient = tf.matmul(tf.transpose(inputs), upstream)
        bias_gradient = upstream
        return inputs_gradient, weights_gradient, bias_gradient
    
    return z, grad

In [42]:
'''
Copyright (C) 2020. Huawei Technologies Co., Ltd. All rights reserved.
This program is free software; you can redistribute it and/or modify
it under the terms of BSD 3-Clause License.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
BSD 3-Clause License for more details.
'''
# import torch
# import torch.nn as nn
# import numpy as np
# from torch.autograd import Function
# import math



# loss           = mean_(layer_output[:,:,:,filter_index])
# grads          = K.gradients(loss,model_input)[0]
# grads          = grads / (sqrt_(mean_(square_(grads))) + 1e-5)
# iterate        = K.function([model_input], [loss,grads])
# input_img_data = np.random.random((1,size,size,3))*20+128

# class adder(Function):

@tf.function
class my_adder:
    @staticmethod
    def forward(ctx, W_col, X_col):
        ctx.save_for_backward(W_col,X_col)
        output = -(W_col.unsqueeze(2)-X_col.unsqueeze(0)).abs().sum(1)
        return output

    @staticmethod
    def backward(ctx,grad_output):
        W_col,X_col = ctx.saved_tensors
        grad_W_col = ((X_col.unsqueeze(0)-W_col.unsqueeze(2))*grad_output.unsqueeze(1)).sum(2)
        grad_W_col = grad_W_col/grad_W_col.norm(p=2).clamp(min=1e-12)*math.sqrt(W_col.size(1)*W_col.size(0))/5
        grad_X_col = (-(X_col.unsqueeze(0)-W_col.unsqueeze(2)).clamp(-1,1)*grad_output.unsqueeze(1)).sum(0)
        
        return grad_W_col, grad_X_col

def adder2d_function(X, W, stride=1, padding=0):
    n_filters, d_filter, h_filter, w_filter = W.size()
    n_x, d_x, h_x, w_x = X.size()

    h_out = (h_x - h_filter + 2 * padding) / stride + 1
    w_out = (w_x - w_filter + 2 * padding) / stride + 1

    h_out, w_out = int(h_out), int(w_out)
    X_col = torch.nn.functional.unfold(X.view(1, -1, h_x, w_x), h_filter, dilation=1, padding=padding, stride=stride).view(n_x, -1, h_out*w_out)
    X_col = X_col.permute(1,2,0).contiguous().view(X_col.size(1),-1)
    W_col = W.view(n_filters, -1)
    
    out = adder.apply(W_col,X_col)
    
    out = out.view(n_filters, h_out, w_out, n_x)
    out = out.permute(3, 0, 1, 2).contiguous()
    
    return out


    
class adder2d(tf.keras.layers.Layer):
    def __init__(self,input_channel,output_channel,kernel_size, stride=1, padding=0, bias = False):
        super(adder2d, self).__init__()
        self.stride = stride
        self.padding = padding
        self.input_channel = input_channel
        self.output_channel = output_channel
        self.kernel_size = kernel_size
        self.adder = torch.nn.Parameter(nn.init.normal_(torch.randn(output_channel,input_channel,kernel_size,kernel_size)))
        self.bias = bias
        if bias:
            self.b = torch.nn.Parameter(nn.init.uniform_(torch.zeros(output_channel)))

    def forward(self, x):
        output = adder2d_function(x,self.adder, self.stride, self.padding)
        if self.bias:
            output += self.b.unsqueeze(0).unsqueeze(2).unsqueeze(3)
        
        return output
    

In [44]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Function
import math

def adder2d_function(X, W, stride=1, padding=0):
    n_filters, d_filter, h_filter, w_filter = W.size()
    n_x, d_x, h_x, w_x = X.size()

    h_out = (h_x - h_filter + 2 * padding) / stride + 1
    w_out = (w_x - w_filter + 2 * padding) / stride + 1

    h_out, w_out = int(h_out), int(w_out)
    X_col = torch.nn.functional.unfold(X.view(1, -1, h_x, w_x), h_filter, dilation=1, padding=padding, stride=stride).view(n_x, -1, h_out*w_out)
    X_col = X_col.permute(1,2,0).contiguous().view(X_col.size(1),-1)
    W_col = W.view(n_filters, -1)
    
    out = adder.apply(W_col,X_col)
    
    out = out.view(n_filters, h_out, w_out, n_x)
    out = out.permute(3, 0, 1, 2).contiguous()
    
    return out

class adder(Function):
    @staticmethod
    def forward(ctx, W_col, X_col):
        ctx.save_for_backward(W_col,X_col)
        output = -(W_col.unsqueeze(2)-X_col.unsqueeze(0)).abs().sum(1)
        return output

    @staticmethod
    def backward(ctx,grad_output):
        W_col,X_col = ctx.saved_tensors
        grad_W_col = ((X_col.unsqueeze(0)-W_col.unsqueeze(2))*grad_output.unsqueeze(1)).sum(2)
        grad_W_col = grad_W_col/grad_W_col.norm(p=2).clamp(min=1e-12)*math.sqrt(W_col.size(1)*W_col.size(0))/5
        grad_X_col = (-(X_col.unsqueeze(0)-W_col.unsqueeze(2)).clamp(-1,1)*grad_output.unsqueeze(1)).sum(0)
        
        return grad_W_col, grad_X_col
    
class adder2d(nn.Module):

    def __init__(self,input_channel,output_channel,kernel_size, stride=1, padding=0, bias = False):
        super(adder2d, self).__init__()
        self.stride = stride
        self.padding = padding
        self.input_channel = input_channel
        self.output_channel = output_channel
        self.kernel_size = kernel_size
        self.adder = torch.nn.Parameter(nn.init.normal_(torch.randn(output_channel,input_channel,kernel_size,kernel_size)))
        self.bias = bias
        if bias:
            self.b = torch.nn.Parameter(nn.init.uniform_(torch.zeros(output_channel)))

    def forward(self, x):
        output = adder2d_function(x,self.adder, self.stride, self.padding)
        if self.bias:
            output += self.b.unsqueeze(0).unsqueeze(2).unsqueeze(3)
        
        return output

In [67]:
#adder2d_function(F_torch,X_torch)
n_filters, d_filter, h_filter, w_filter = F_torch.size()
n_x, d_x, h_x, w_x = X_torch.size()

p=0
stride=1
padding=0
h_out = (h_x - h_filter + 2 * p) / stride + 1
w_out = (w_x - w_filter + 2 * p) / stride + 1

h_out, w_out = int(h_out), int(w_out)
X_col = torch.nn.functional.unfold(X_torch.view(1, -1, h_x, w_x), h_filter, dilation=1, padding=padding, stride=stride).view(n_x, -1, h_out*w_out)
X_col = X_col.permute(1,2,0).contiguous().view(X_col.size(1),-1)
W_col = F_torch.view(n_filters, -1)

out = adder.apply(W_col,X_col)

#out = out.view(n_filters, h_out, w_out, n_x)
#out = out.permute(3, 0, 1, 2).contiguous()



RuntimeError: ignored

In [ ]:
'''Equation 2'''
# m = row (y term)
# n = column (x term)
def Y_adder(X, F, m, n, t, similarity_f=L1):
    # F.shape returns (#filters, #channels, #rows, #columns)
    # X.shape returns (#channels, #rows, #columns)
    # t specifies filter #
    sum_ = 0
    filters = F.shape[0]
    in_ = F.shape[0]
    c_out,k, k, f_channels = F.shape
    n,    W, H, X_channels =X.shape
    assert f_channels == X_channels
    c_in = f_channels
    
    #_, c_in, d, _ = F.shape     
    #for k in range(c_in):
    #    for j in range(d):
    #        for i in range(d):
    #            sum_ += similarity_f(X[k, m+i, n+j], F[i, j, k, t])
    #return sum_

In [11]:
f_n, f_d, f_h, f_w = 5,  3, 3, 3
X_n, X_d, X_h, X_w = 10, 3, 5, 5

F_rand  = np.random.random((f_n,f_d,f_h,f_w))
F_torch = torch.tensor(F_rand)
F_tf    = tf.convert_to_tensor(F_rand)

X_rand  = np.random.random((X_n,X_d,X_h,X_w))
X_torch = torch.tensor(X_rand)
X_tf    = tf.convert_to_tensor(X_rand)

In [13]:
padding='VALID'
if padding=='VALID':
    p=0
else:
    p=1
W,X = F_tf, X_tf
n_filters, d_filter, h_filter, w_filter = W.shape
n_x, d_x, h_x, w_x = X.shape
stride=1


h_out = (h_x - h_filter + 2 * p) / stride + 1
w_out = (w_x - w_filter + 2 * p) / stride + 1
k_size=3
stride=1
h_out, w_out = int(h_out), int(w_out)

X_col = tf.image.extract_patches(np.rollaxis(X,1,3), 
                                sizes=[1,k_size,k_size,1], 
                                strides=[1,stride,stride,1], 
                                rates=[1,1,1,1], 
                                padding='VALID') #'VALID' = 0, 'SAME' = however many to not change shape

X_col = X_col.reshape((np.prod(X_col.shape[:-1]),X_col.shape[-1]))

W_col = W.reshape(n_filters,-1)

#out = adder.apply(torch.tensor(W_col.numpy()),torch.tensor(X_col.numpy()))
# out = out.view(n_filters, h_out, w_out, n_x)
# out = out.permute(3, 0, 1, 2).contiguous()

In [160]:
def get_conv_windows(images, k, s, p):
    c_in = images.shape[1]
    #c_out = filters.shape[1]
    padding='VALID' if p==0 else 'SAME'
    images_ = tf.image.extract_patches(images=np.rollaxis(images,1,4), 
                                    sizes=[1, k, k, 1], 
                                    strides=[1, 1, 1, 1], 
                                    rates=[1, 1, 1, 1], 
                                    padding='VALID')

    #display(images_)
    images_ = images_.transpose((0,3,1,2))
    d_0,d_1,d_2,d_3 = images_.shape


    n_groups = int(d_1/c_in)

    images_ = images_.reshape((d_0,n_groups,c_in,d_2,d_3))

    # # shape: n_images, new H * new W, n_channels, k, k
    print(images_.shape)

    return images_


n_images, n_channels, H, W = 1, 3, 5, 5
n_filters = 5
k = 3

images  = np.random.randint(low=0,high=10,size = (n_images,  7, H, W)) # (1,3,5,5)
# weights = np.random.randint(low=0,high=10,size = (n_filters, n_channels, k, k)) # (5,3,3,3)
new_images = get_conv_windows(images,3,1,0)



(1, 9, 7, 3, 3)


In [162]:
def get_windows(tensor_of_images,k_d=3,k_w=3,k_h=3,stride=0,padding=0):
    imgs = np.pad(tensor_of_images,pad_width=padding)
    n = len(imgs)
    return skimage.util.view_as_windows(imgs, (n,k_d,k_w,k_h), step=stride)



import skimage
skimage.util.view_as_windows(images, (1,3,3,3), step=1)

In [171]:
images

array([[[[6, 7, 0, 7, 6],
         [3, 4, 3, 4, 4],
         [6, 4, 4, 5, 3],
         [4, 3, 9, 2, 0],
         [5, 6, 4, 0, 5]],

        [[1, 2, 7, 3, 7],
         [6, 9, 6, 9, 9],
         [8, 4, 4, 9, 4],
         [6, 3, 8, 3, 5],
         [6, 1, 4, 4, 6]],

        [[2, 1, 2, 5, 1],
         [6, 0, 1, 2, 4],
         [0, 2, 6, 4, 0],
         [9, 1, 6, 6, 4],
         [6, 5, 2, 7, 8]],

        [[8, 3, 5, 6, 5],
         [8, 0, 8, 2, 4],
         [5, 0, 7, 2, 1],
         [4, 0, 7, 7, 3],
         [8, 2, 6, 9, 7]],

        [[9, 3, 9, 1, 7],
         [6, 8, 7, 5, 6],
         [2, 4, 1, 5, 7],
         [2, 6, 0, 5, 9],
         [1, 1, 3, 3, 3]],

        [[2, 4, 9, 6, 3],
         [9, 6, 4, 3, 4],
         [9, 4, 7, 1, 0],
         [9, 5, 5, 0, 5],
         [9, 1, 1, 5, 3]],

        [[3, 9, 4, 1, 0],
         [4, 7, 0, 8, 7],
         [6, 4, 6, 6, 8],
         [2, 4, 0, 5, 8],
         [0, 5, 6, 5, 5]]]])

In [169]:
skimage.util.view_as_windows(images, (1,3,3,3), step=1)

array([[[[[[[[6, 7, 0],
             [3, 4, 3],
             [6, 4, 4]],

            [[1, 2, 7],
             [6, 9, 6],
             [8, 4, 4]],

            [[2, 1, 2],
             [6, 0, 1],
             [0, 2, 6]]]],



          [[[[7, 0, 7],
             [4, 3, 4],
             [4, 4, 5]],

            [[2, 7, 3],
             [9, 6, 9],
             [4, 4, 9]],

            [[1, 2, 5],
             [0, 1, 2],
             [2, 6, 4]]]],



          [[[[0, 7, 6],
             [3, 4, 4],
             [4, 5, 3]],

            [[7, 3, 7],
             [6, 9, 9],
             [4, 9, 4]],

            [[2, 5, 1],
             [1, 2, 4],
             [6, 4, 0]]]]],




         [[[[[3, 4, 3],
             [6, 4, 4],
             [4, 3, 9]],

            [[6, 9, 6],
             [8, 4, 4],
             [6, 3, 8]],

            [[6, 0, 1],
             [0, 2, 6],
             [9, 1, 6]]]],



          [[[[4, 3, 4],
             [4, 4, 5],
             [3, 9, 2]],

            [[9, 6, 

In [ ]:
3 image channels --> 1,27,3,3
1 image channel --> 1,9,3,3

In [144]:
images

array([[[[3, 6, 2, 3, 7],
         [2, 0, 1, 2, 3],
         [1, 3, 8, 8, 7],
         [9, 0, 9, 7, 8],
         [1, 3, 8, 8, 5]],

        [[4, 9, 8, 7, 0],
         [2, 5, 4, 1, 9],
         [5, 4, 7, 2, 4],
         [3, 2, 3, 3, 6],
         [1, 0, 5, 4, 3]],

        [[9, 6, 2, 2, 5],
         [0, 8, 3, 2, 3],
         [9, 3, 4, 7, 3],
         [1, 2, 9, 8, 3],
         [8, 0, 8, 0, 5]]]])

In [145]:
new_images

<tf.Tensor: shape=(1, 9, 3, 3, 3), dtype=int64, numpy=
array([[[[[3, 6, 2],
          [2, 0, 1],
          [1, 3, 8]],

         [[4, 9, 8],
          [2, 5, 4],
          [5, 4, 7]],

         [[9, 6, 2],
          [0, 8, 3],
          [9, 3, 4]]],


        [[[6, 2, 3],
          [0, 1, 2],
          [3, 8, 8]],

         [[9, 8, 7],
          [5, 4, 1],
          [4, 7, 2]],

         [[6, 2, 2],
          [8, 3, 2],
          [3, 4, 7]]],


        [[[2, 3, 7],
          [1, 2, 3],
          [8, 8, 7]],

         [[8, 7, 0],
          [4, 1, 9],
          [7, 2, 4]],

         [[2, 2, 5],
          [3, 2, 3],
          [4, 7, 3]]],


        [[[2, 0, 1],
          [1, 3, 8],
          [9, 0, 9]],

         [[2, 5, 4],
          [5, 4, 7],
          [3, 2, 3]],

         [[0, 8, 3],
          [9, 3, 4],
          [1, 2, 9]]],


        [[[0, 1, 2],
          [3, 8, 8],
          [0, 9, 7]],

         [[5, 4, 1],
          [4, 7, 2],
          [2, 3, 3]],

         [[8, 3, 2],
      

In [119]:
n_images, n_values, n_channels_in, new_H, new_W = new_images.shape

p = 0
s = 1
W_ = int((W+2*p-k)/s+1)
H_ = int((H+2*p-k)/s+1)
n_channels_out = int((n_channels_in + 2*p - n_channels_in)+1)




out = np.zeros((len(n_filters),n_channels_out, W_, H_))


for f in n_filters:
    for c in range(n_channels_out):
        #print(f'Image {i+1}\n')
        for group in range(n_groups):
            image_group = final_image[i][group]
            added = (image_group + weight)
            all_added=[]
            for a in added:
                all_added.append(K.sum(a))
            print(sum(all_added))

            #print(added.shape)
            #print(added)
            #print(K.sum(added,axis=2))
            print('***')
            #added = group + weights

# tf.Tensor(
# [[[ 1 11  3]
#   [ 5  5  9]
#   [ 8  7 18]]



#  [[14 12  1]
#   [ 9  8 12]
#   [10  3  4]]

#  [[ 5 13 13]
#   [ 7 14  2]
#   [15  6 11]]], shape=(3, 3, 3), dtype=int64)

Image 1

tf.Tensor(226, shape=(), dtype=int64)
***
tf.Tensor(211, shape=(), dtype=int64)
***
tf.Tensor(211, shape=(), dtype=int64)
***
tf.Tensor(235, shape=(), dtype=int64)
***
tf.Tensor(235, shape=(), dtype=int64)
***
tf.Tensor(233, shape=(), dtype=int64)
***
tf.Tensor(230, shape=(), dtype=int64)
***
tf.Tensor(236, shape=(), dtype=int64)
***
tf.Tensor(245, shape=(), dtype=int64)
***


In [45]:
n_g

0

In [28]:
#final_img_shape = [1,27,3,3]

final_img.reshape(d_0,n_groups,n_channels,d_2,d_3)

kernel_size=3
n_filters=5
# weights = np.random.randint(low=0,high=10,size=(n_filters,n_channels,kernel_size,kernel_size))
#n_channels=3
#h,w=5,5
#image = np.random.randint(low=0,high=10,size=(1,n_channels,h,w))

In [34]:
final_img[0,:,:,:]# + weights[0]

<tf.Tensor: shape=(27, 3, 3), dtype=int64, numpy=
array([[[0, 3, 3],
        [5, 2, 9],
        [3, 4, 9]],

       [[9, 8, 1],
        [2, 8, 5],
        [8, 3, 2]],

       [[5, 5, 4],
        [5, 5, 0],
        [7, 4, 7]],

       [[3, 3, 0],
        [2, 9, 9],
        [4, 9, 4]],

       [[8, 1, 2],
        [8, 5, 1],
        [3, 2, 6]],

       [[5, 4, 0],
        [5, 0, 0],
        [4, 7, 7]],

       [[3, 0, 4],
        [9, 9, 7],
        [9, 4, 2]],

       [[1, 2, 6],
        [5, 1, 8],
        [2, 6, 7]],

       [[4, 0, 6],
        [0, 0, 1],
        [7, 7, 1]],

       [[5, 2, 9],
        [3, 4, 9],
        [5, 8, 5]],

       [[2, 8, 5],
        [8, 3, 2],
        [0, 0, 7]],

       [[5, 5, 0],
        [7, 4, 7],
        [8, 9, 5]],

       [[2, 9, 9],
        [4, 9, 4],
        [8, 5, 4]],

       [[8, 5, 1],
        [3, 2, 6],
        [0, 7, 5]],

       [[5, 0, 0],
        [4, 7, 7],
        [9, 5, 7]],

       [[9, 9, 7],
        [9, 4, 2],
        [5, 4, 2]],

      

In [32]:
weights

array([[[[1, 8, 0],
         [0, 3, 0],
         [5, 3, 9]],

        [[5, 4, 0],
         [7, 0, 7],
         [2, 0, 2]],

        [[0, 8, 9],
         [2, 9, 2],
         [8, 2, 4]]],


       [[[9, 3, 9],
         [6, 5, 7],
         [7, 6, 6]],

        [[3, 2, 8],
         [5, 2, 5],
         [2, 4, 2]],

        [[3, 0, 1],
         [7, 5, 9],
         [4, 5, 2]]],


       [[[0, 0, 0],
         [8, 9, 4],
         [2, 1, 1]],

        [[4, 9, 0],
         [4, 1, 2],
         [1, 3, 1]],

        [[4, 8, 6],
         [0, 4, 9],
         [7, 8, 2]]],


       [[[7, 4, 0],
         [4, 1, 0],
         [7, 7, 6]],

        [[2, 4, 0],
         [7, 4, 4],
         [3, 0, 0]],

        [[6, 1, 8],
         [2, 7, 3],
         [2, 0, 6]]],


       [[[3, 9, 8],
         [1, 4, 6],
         [9, 7, 0]],

        [[3, 2, 5],
         [3, 9, 4],
         [2, 5, 1]],

        [[2, 8, 3],
         [9, 3, 6],
         [5, 6, 5]]]])